## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-23-02-27-00 +0000,wsj,The U.S.’s pursuit of oil tankers around Venez...,https://www.wsj.com/politics/national-security...
1,2025-12-23-02-26-00 +0000,wsj,Yen Strengthens After Japan’s Katayama Warns o...,https://www.wsj.com/finance/currencies/yen-mos...
2,2025-12-23-02-13-29 +0000,nypost,Pint-sized perps lead cops on frantic chase — ...,https://nypost.com/2025/12/22/us-news/pint-siz...
3,2025-12-23-02-08-03 +0000,wapo,Education Dept. to examine safety procedures a...,https://www.washingtonpost.com/education/2025/...
4,2025-12-23-02-08-00 +0000,wsj,Trump Appointment of Greenland Envoy Draws Ang...,https://www.wsj.com/world/trump-appointment-of...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2405/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,40
6,new,17
60,up,14
361,christmas,12
684,general,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2025-12-23-02-27-00 +0000,wsj,The U.S.’s pursuit of oil tankers around Venez...,https://www.wsj.com/politics/national-security...,123
41,2025-12-23-00-25-00 +0000,wsj,President Trump said he wants to pressure Amer...,https://www.wsj.com/politics/policy/trump-targ...,101
177,2025-12-22-16-52-46 +0000,nypost,Trump admin scraps 5 offshore wind farms — inc...,https://nypost.com/2025/12/22/us-news/trump-ad...,91
141,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...,90
100,2025-12-22-21-55-00 +0000,wsj,A coalition of Democratic state attorneys gene...,https://www.wsj.com/politics/policy/democratic...,76


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,123,2025-12-23-02-27-00 +0000,wsj,The U.S.’s pursuit of oil tankers around Venez...,https://www.wsj.com/politics/national-security...
272,69,2025-12-22-08-33-37 +0000,nypost,Russian general Fanil Sarvarov dead in Moscow ...,https://nypost.com/2025/12/22/world-news/russi...
205,53,2025-12-22-14-47-26 +0000,latimes,Larry Ellison pledges $40-billion personal gua...,https://www.latimes.com/entertainment-arts/bus...
141,50,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...
119,46,2025-12-22-20-39-00 +0000,wsj,Former Vice President Mike Pence’s political g...,https://www.wsj.com/politics/elections/top-her...
229,36,2025-12-22-12-32-29 +0000,nypost,Australian PM Anthony Albanese apologizes to J...,https://nypost.com/2025/12/22/world-news/austr...
204,36,2025-12-22-14-49-17 +0000,nypost,Jimmy Kimmel to make Christmas Day address on ...,https://nypost.com/2025/12/22/media/jimmy-kimm...
144,35,2025-12-22-19-01-17 +0000,nyt,"What to Know About Louisiana Gov. Jeff Landry,...",https://www.nytimes.com/2025/12/22/us/jeff-lan...
177,34,2025-12-22-16-52-46 +0000,nypost,Trump admin scraps 5 offshore wind farms — inc...,https://nypost.com/2025/12/22/us-news/trump-ad...
8,33,2025-12-23-01-58-49 +0000,nyt,Turmoil at CBS News After Bari Weiss Pulls a ‘...,https://www.nytimes.com/2025/12/22/business/me...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
